# FinTech A/B Test – Signup & Activation Funnel

### Objective
Evaluate whether version B improves the user journey compared to version A:

- Signup Completion  
- Deposit Rate  
- First Transaction Rate  

In [13]:
import pandas as pd
df = pd.read_csv("fintech_ab_test_dataset.csv")
df.head()

,user_id,version,viewed_signup,clicked_cta,completed_signup,made_deposit,first_transaction
0,1,A,1,0,0,0,0
1,2,B,1,0,0,0,0
2,3,B,1,1,0,0,0
3,4,B,1,0,0,0,0
4,5,A,1,0,0,0,0


In [18]:
summary = df.groupby("version").agg(
    users=("user_id", "count"),
    views=("viewed_signup", "sum"),
    clicks=("clicked_cta", "sum"),
    signups=("completed_signup", "sum"),
    deposits=("made_deposit", "sum"),
    first_txns=("first_transaction", "sum"))
summary

,users,views,clicks,signups,deposits,first_txns
version,,,,,,
A,25053,25053,7949,2496,980,533
B,24947,24947,9555,3586,1494,860


In [20]:
summary["ctr"] = summary["clicks"] / summary["views"]                 # Click-through rate
summary["signup_cr"] = summary["signups"] / summary["views"]          # Signup conversion
summary["deposit_cr"] = summary["deposits"] / summary["views"]        # Deposit rate
summary["txn_cr"] = summary["first_txns"] / summary["views"]          # First transaction rate
summary

,users,views,clicks,signups,deposits,first_txns,ctr,signup_cr,deposit_cr,txn_cr
version,,,,,,,,,,
A,25053,25053,7949,2496,980,533,0.317287,0.099629,0.039117,0.021275
B,24947,24947,9555,3586,1494,860,0.383012,0.143745,0.059887,0.034473


## Key Metrics

### Signup Conversion
- A: **9.96%**
- B: **14.37%**
- Lift: **44.28%**
### Deposit Rate
- A: **3.9117%**
- B: **5.9887	%**
- Lift: **53.09%**
### First Transaction Rate
- A: **2.12%**
- B: **3.44%**
- Lift: **62.03%**

In [17]:
# signup
stat_signup, p_signup = proportions_ztest(summary["signups"], summary["views"])

# deposit
stat_dep, p_dep = proportions_ztest(summary["deposits"], summary["views"])

# transaction
stat_txn, p_txn = proportions_ztest(summary["first_txns"], summary["views"])

stat_signup, p_signup, stat_dep, p_dep, stat_txn, p_txn


(np.float64(-15.089512323679163),
 np.float64(1.8984023980700633e-51),
 np.float64(-10.70761436112739),
 np.float64(9.374646799256595e-27),
 np.float64(-8.966300774535338),
 np.float64(3.0663984898034186e-19))

### 🔬 Statistical Significance (Z-test)

- **Signup p-value:** 1.8984e-51 
- **Deposit p-value:** 9.37446e-27  
- **First Transaction p-value:**  3.06639e-19  

> All p-values are **far below 0.05**, meaning the improvement in version B is **statistically significant** across all funnel stages.


In [38]:
A = summary.loc["A"]
B = summary.loc["B"]

lift_signup = (B["signup_cr"] - A["signup_cr"]) / A["signup_cr"]
lift_deposit = (B["deposit_cr"] - A["deposit_cr"]) / A["deposit_cr"]
lift_txn = (B["txn_cr"] - A["txn_cr"]) / A["txn_cr"]

lift_signup, lift_deposit, lift_txn


(np.float64(0.4428032621465198),
 np.float64(0.5309673650997258),
 np.float64(0.6203642528915524))

## Business Conclusion

Version **B** significantly improves both conversion and financial activation:
- More users sign up (+44%)
- More users deposit (+53%)
- More users complete a first transaction (+62%)

This indicates higher **LTV**, lower **CAC payback**, and overall better product performance.

**Recommendation:** roll out Version B for all new users.
